In [ ]:
from selenium.webdriver import Chrome, Firefox

In [ ]:
# create an instance of web driver
browser = Firefox()

In [ ]:
browser.get('https://en.wikipedia.org/wiki/Main_Page')
languages = browser.find_elements_by_class_name('interlanguage-link')

In [ ]:
print(languages[0])

In [ ]:
language_names = [language.find_element_by_css_selector('a').text for language in languages]
links = [language.find_element_by_css_selector('a').get_attribute('href') for language in languages]

In [ ]:
scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
browser.execute_script(scroll_down)

In [ ]:
content_element = browser.find_element_by_id('n-contents').find_element_by_css_selector('a')
content_element.click()

In [ ]:
from urllib.request import urlretrieve
from uuid import uuid4
browser.get('https://en.wikipedia.org/wiki/Main_Page')
images = browser.find_elements_by_css_selector('img')
src_list = [img.get_attribute('src') for img in images]
for src in src_list:
    uuid = uuid4()
    urlretrieve(src, f"wiki_images/{uuid}.jpg")

In [ ]:
import time
urls = links[0:3]
for url in urls:
    browser.get(url)
    time.sleep(3)

In [ ]:

# closing the web driver

browser.close()